In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [18]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [19]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # 定义网络层：输入层到隐藏层，隐藏层到输出层
        self.fc1 = nn.Linear(28 * 28, 128)  # 输入层到隐藏层
        self.fc2 = nn.Linear(128, 10)     # 隐藏层到输出层
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)             # 展平输入
        x = torch.relu(self.fc1(x))       # 隐藏层+ReLU激活
        x = self.fc2(x)                   # 输出层（未加softmax）
        return x

# 实例化模型
model = SimpleNN()
criterion = nn.CrossEntropyLoss()        # 交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam优化器

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    # 训练循环（修复了此处缩进）
    for images, labels in train_loader:
        optimizer.zero_grad()            # 梯度清零
        outputs = model(images)          # 前向传播
        loss = criterion(outputs, labels)# 计算损失
        loss.backward()                  # 反向传播
        optimizer.step()                 # 更新参数
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

# 测试循环
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # 获取预测类别
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total  # 移至循环外计算最终准确率
print(f"Accuracy on test set: {accuracy:.2f}%")

Epoch 1/5, Loss: 0.3861
Epoch 2/5, Loss: 0.1989
Epoch 3/5, Loss: 0.1444
Epoch 4/5, Loss: 0.1139
Epoch 5/5, Loss: 0.0973
Accuracy on test set: 96.73%
